In [ ]:
import geopandas as gpd
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_fixed, retry_if_exception_type
from sqlalchemy.exc import OperationalError, SQLAlchemyError
from dotenv import load_dotenv
import os
import pandas as pd

In [ ]:
load_dotenv()
engine = create_engine(os.getenv("SUPABASE_DB_URL"),connect_args={"options": "-c statement_timeout=300000"})
print(engine)

In [ ]:
# === PARAMETER UPLOAD ===
TABLE_NAME = "id_jkt_geohash7"
CHUNKSIZE = 1000  # baris per batch
MAX_RETRY = 3     # jumlah percobaan ulang

In [ ]:
# === BACA DATA ===
gdf = gpd.read_parquet(r"C:\Users\hardy\Documents\Python\Data Portofolio\data-portofolio-hardy\projects\optimalization-pickup-ridehailing\data\id_adm_1.parquet").to_crs(epsg=4326)

In [ ]:
# === BACA DATA GEOJSON ===
gdf = gpd.read_file(r"C:\Users\hardy\Documents\Python\Data Portofolio\data-portofolio-hardy\projects\optimalization-pickup-ridehailing\data\id_jkt_geohash7.geojson").to_crs(epsg=4326)

In [ ]:
# === FUNGSI UPLOAD DENGAN RETRY ===
@retry(
    stop=stop_after_attempt(MAX_RETRY),
    wait=wait_fixed(3),
    retry=retry_if_exception_type((OperationalError, SQLAlchemyError)),
    reraise=True
)
def upload_chunk(chunk, engine, table_name):
    chunk.to_postgis(table_name, engine, if_exists="append", index=False)

In [ ]:
# === PENGOSONGAN TABEL JIKA ADA SEBELUMNYA ===
with engine.begin() as conn:
    conn.execute(f'DROP TABLE IF EXISTS "{TABLE_NAME}";')

In [ ]:
# === UPLOAD DALAM BATCH DENGAN PROGRESS BAR ===
num_chunks = (len(gdf) // CHUNKSIZE) + 1
for i in tqdm(range(num_chunks), desc="Uploading", unit="chunk"):
    start = i * CHUNKSIZE
    end = min((i + 1) * CHUNKSIZE, len(gdf))
    chunk = gdf.iloc[start:end]
    
    if i == 0:
        # batch pertama: buat tabel
        chunk.to_postgis(TABLE_NAME, engine, if_exists="replace", index=False, dtype={"geometry": Geometry("MULTIPOLYGON", srid=4326)})
    else:
        upload_chunk(chunk, engine, TABLE_NAME)

print("✅ Upload selesai tanpa error.")

In [ ]:
gdf = gpd.read_postgis(
    sql="SELECT * FROM id_jkt_geohash7;",
    con=engine,
    geom_col="geometry"  # ganti sesuai nama kolom geometri
)


In [ ]:
gdf.explore()